This notbook is a basic application of optuna on one wifi to tune the parameters of LGBM regressor. It currently only tunes based on 1 wifi trian set.

Based on the example: https://github.com/optuna/optuna/blob/master/examples/lightgbm_simple.py

Also credit to @devinanzelmo for the beginning notebook and dataset for wifi

In [ ]:
import lightgbm as lgb
import numpy as np
import pandas as pd
import lightgbm as lgb
import glob
import os
import sklearn.datasets
import sklearn.metrics
from sklearn.model_selection import train_test_split

import optuna


In [ ]:
feature_dir = "../input/indoor-wifi-features/"

In [ ]:
# the metric used in this competition
def comp_metric(xhat, yhat, fhat, x, y, f):
    intermediate = np.sqrt(np.power(xhat - x,2) + np.power(yhat-y,2)) + 15 * np.abs(fhat-f)
    return intermediate.sum()/xhat.shape[0]

# get our train and test files
train_files = sorted(glob.glob(os.path.join(feature_dir, '*_train.csv')))
print (train_files)
test_files = sorted(glob.glob(os.path.join(feature_dir, '*_test.csv')))
ssubm = pd.read_csv('../input/indoor-location-navigation/sample_submission.csv', index_col=0)

In [ ]:
file = "../input/indoor-wifi-features/5a0546857ecc773753327266_1000_train.csv"
data = pd.read_csv(file, index_col=0)
data

In [ ]:

# FYI: Objective functions can take additional arguments
# (https://optuna.readthedocs.io/en/stable/faq.html#objective-func-additional-args).
def objective(trial):
    #data, target = sklearn.datasets.load_breast_cancer(return_X_y=True)
    file = "../input/indoor-wifi-features/5a0546857ecc773753327266_1000_train.csv"
    data = pd.read_csv(file, index_col=0)
    
    x_train = data.iloc[:,:-3]
    y_trainy = data.iloc[:,-2]
    
    train_x, valid_x, train_y, valid_y = train_test_split(x_train, y_trainy, test_size=0.25)
    dtrain = lgb.Dataset(train_x, label=train_y)
    '''
    param = {
        "objective": "regression",
        "metric": "binary_logloss",
        "verbosity": -1,
        "boosting_type": "gbdt",
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-8, 10.0, log=True),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-8, 10.0, log=True),
        "n_estimators": trial.suggest_int("n_estimators", 50, 200),
        "num_leaves": trial.suggest_int("num_leaves", 2, 256),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.4, 1.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.4, 1.0),
        "bagging_freq": trial.suggest_int("bagging_freq", 1, 7),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
    }
    '''
    param = {"objective": "regression","metric": "l2","verbosity": -1,"boosting_type": "gbdt","n_estimators": trial.suggest_int("n_estimators", 50, 200),"num_leaves": trial.suggest_int("num_leaves", 10, 150)}

    gbm = lgb.train(param, dtrain)
    preds = gbm.predict(valid_x)
    pred_labels = np.rint(preds)
    accuracy = sklearn.metrics.mean_squared_error(valid_y, pred_labels)
    return accuracy


if __name__ == "__main__":
    study = optuna.create_study(direction="minimize")
    ## increase n_trails to test more params
    study.optimize(objective, n_trials=5)

    print("Number of finished trials: {}".format(len(study.trials)))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: {}".format(trial.value))

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))
